In [1]:
%pip install openai faiss-cpu numpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [2]:
# Get the openai secret key:
import getpass

secret_key = getpass.getpass('Please enter your openai key: ')

Please enter your openai key: ··········


In [3]:
from openai import OpenAI
import numpy as np
import faiss
import re

client = OpenAI(api_key=secret_key)
client.api_key = secret_key

# Function to get the vector embedding for a given text
def get_vector_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    embeddings = response.data[0].embedding
    return embeddings

get_vector_embeddings("Your text string goes here")

[-0.007611386943608522,
 -0.005377683322876692,
 0.011407003737986088,
 -0.024762200191617012,
 -0.02490999363362789,
 0.039796870201826096,
 -0.010325420647859573,
 -0.009411785751581192,
 -0.013294734992086887,
 -0.009841731749475002,
 -0.011736180633306503,
 0.008229434490203857,
 -0.014295702800154686,
 0.007765898481011391,
 0.01000296138226986,
 -0.004937660414725542,
 0.02282744273543358,
 -0.0016215344658121467,
 0.015330259688198566,
 -0.010258241556584835,
 0.004890635143965483,
 0.012240023352205753,
 0.004742841236293316,
 0.010547111742198467,
 -0.006593624129891396,
 -0.0003955166903324425,
 0.005703501868993044,
 -0.012811045162379742,
 0.016364818438887596,
 0.004437176510691643,
 0.006371933035552502,
 -0.006798520218580961,
 -0.015303388237953186,
 -0.006170396227389574,
 -0.018178651109337807,
 0.0037049248348921537,
 0.0036108740605413914,
 -0.019441617652773857,
 0.030311189591884613,
 -0.006953032221645117,
 0.00816897302865982,
 0.009492400102317333,
 -0.00093126

In [4]:
get_vector_embeddings("I'm Kempsly, a machine learning engineer!")

[-0.013057257048785686,
 0.002226352458819747,
 0.013578744605183601,
 -0.03885754197835922,
 -0.018479393795132637,
 0.029310299083590508,
 0.015096408315002918,
 0.012682855129241943,
 -0.004288904368877411,
 -0.022945471107959747,
 0.011492792516946793,
 0.005027678795158863,
 -0.014187146909534931,
 -0.01600566878914833,
 0.01213462371379137,
 -0.0060539403930306435,
 0.0209798626601696,
 -0.003296071430668235,
 0.007574947085231543,
 -0.0039245313964784145,
 -0.018720081076025963,
 -0.00887198094278574,
 0.020538603886961937,
 -0.019923515617847443,
 -0.013351429253816605,
 0.0024369533639401197,
 0.00990158598870039,
 -0.0069799162447452545,
 -0.012234910391271114,
 -0.017770705744624138,
 0.018853796645998955,
 -0.013759260065853596,
 -0.012950284406542778,
 -0.014320862479507923,
 -0.008658037520945072,
 -0.003291057189926505,
 0.0011031476315110922,
 -0.007595004513859749,
 0.03463215008378029,
 -0.023627417162060738,
 0.01686144433915615,
 0.01083759032189846,
 -0.00810980703

In [5]:
import re

def chunk_text(text, window_size=5, stride=2):
    # Tokenize the text into sentences
    sentences = re.split('(?<=[.!?]) +', text.strip())

    # Initialize the list of chunks
    chunks = []

    # Slide the window over the sentences
    for i in range(0, len(sentences) - window_size + 1, stride):
        # Add the current window to the chunks
        chunks.append(' '.join(sentences[i:i + window_size]))

    return chunks

text = """
From Adam West to Robert Pattinson: this is how Batman has changed over the years
Batman is one of the most recognizable superheroes in pop culture, starting with his first appearance in Detective Comics #27 in 1939. He has been listed as one of the most iconic characters in popular culture, and among the greatest comic book superheroes and fictional characters ever created. While Batman is sometimes paired with Superman,

Isabel Cara

00:08 / 00:20
Batman is one of the most recognizable superheroes in pop culture, starting with his first appearance in Detective Comics #27 in 1939. He has been listed as one of the most iconic characters in popular culture, and among the greatest comic book superheroes and fictional characters ever created.

While Batman is sometimes paired with Superman, Wonder Woman, and others from the Justice League, his stories have so much potential that they have been made into movies, tv shows, videogames, toy lines, radio, even musical theatre.

You might find interesting: ‘The Batman’: Villains who would be exceptional candidates for the sequel

ADVERTISING

So, let’s talk about how Batman, in his live-action version, has changed through the years:

The Batman series (1966-1968), played by Adam West
V2qn2wliqvchxfxv6w56zqm6se - from adam west to robert pattinson: this is how batman has changed over the years
This tv show (and a movie in 1966) portrays the character in a camp aesthetic, meaning that it could be appealing because of its bad taste and ironic value (like those “so bad it’s good” movies), but they camp things can also be described as “cheesy”.

This aesthetic was associated with Batman for years after the show ended, and comic creators worked hard to return the character to his darker roots in the decades that followed, culminating in The Dark Knight Returns (1986) by Frank Miller.


Nevertheless, this Batman holds a special place in every fan because references to this show have appeared in almost every Batman movie made after. With references appearing in media that is not related to the character, like the movie Ready Player One, and shows like The Simpsons, SpongeBob SquarePants, and The Fairly OddParents.

While it might not be considered the best version of Batman, it’s the Batman that opened the doors to what we have now.

Batman (1989), played by Michael Keaton
6aheuzaikfealdjwgzy6366xt4 - from adam west to robert pattinson: this is how batman has changed over the years
This was a groundbreaking movie directed by Tim Burton, and starring Jack Nicholson, Michael Keaton, Kim Basinger, Robert Wuhl, Pat Hingle, Billy Dee Williams, Michael Gough, and Jack Palance.

The movie was successful critically and financially, making it the 5th highest-grossing film in history (at the time of its release).

It inspired Batman: The Animated Series (1992-1995), paving the way for the DC Animated Universe. The film has also influenced Hollywood’s modern marketing and development techniques of the superhero film genre.

It created the “Batmania” pop-culture phenomenon and made Batman cool again.

Batman Forever (1995), played by Val Kilmer
47zcqedjfrerpgasyogbpwjsvu - from adam west to robert pattinson: this is how batman has changed over the years
This movie strayed away from the aesthetic of Tim Burton’s Batman, making it more colorful and over-the-top, especially with Jim Carey’s Ridley and Tommy Lee Jones’s Two-Face.

It received mixed reviews from critics, but was a box office success, becoming the 6th highest-grossing film worldwide in 1995. Nevertheless, this movie tends to be ignored currently, with many fans not even remembering that Val Kilmer was Batman.

Batman & Robin (1997), played by George Clooney
3bulsal75ncwzny3xvd2nz33le - from adam west to robert pattinson: this is how batman has changed over the years
This film is regarded as the worst Batman movie and even getting spots as one of the worst movies ever made.


It has become infamous because of the neon design, the Batnipples, and just being a franchise-killing movie (after being considered a box office disappointment, Warner Bros. decided to cancel future Batman films, like Schumacher’s -the director of this movie and Batman Forever- Batman Unchained).

It’s not even considered by fans “so bad it’s good”, and it’s almost always skipped in rewatches of the films.

Batman Begins (2005), played by Christian Bale
Doggryf3mbfklnwfyzr2t7vyoi - from adam west to robert pattinson: this is how batman has changed over the years
The most iconic Batman for the modern audience. Christopher Nolan’s work makes it one of the best superhero movies of all time.

It rebooted the Batman film series, giving it a darker and more realistic tone compared to previous films (especially Schumacher’s job). The film wanted to create an emotional connection with the audience and the Batman and Bruce Wayne identities.

The film became the 9th highest-grossing film of 2005 and was followed by The Dark Knight (2008), and The Dark Knight Rises (2012). Creating The Dark Knight Trilogy, and the reason many DC fans say that DC movies are better than Marvel Films.

Batman v Superman: Dawn of Justice (2016), played by Ben Affleck
R6tzxansdngnra5ee5izdeocva - from adam west to robert pattinson: this is how batman has changed over the years
Affleck is the only actor (so far) to not get his solo movie, and the possibilities of it happening are uncertain, maybe even impossible.

Nevertheless, his role is different from the other Batmans, with him being older and jaded because of all the deaths and destruction he has seen but being hopeful in humanity among the member of the Justice League.

He feels like Batman and Bruce are the same guys, and it really is a shame we couldn’t spend more time with him.

Gotham (2014-2019), played by David Mazouz
Qf7qau37anf75nknc5wwobmnkm - from adam west to robert pattinson: this is how batman has changed over the years
Wait, who? Well… yeah.

The show started following James Gordon in his early days at the Gotham City Police Department following the tragic deaths of Bruce’s parents and started telling the origin stories of some of the most infamous Batman villains.

During the fourth season, Bruce Wayne starts to fight crime as a masked vigilante, and in the fifth (and final) season it shows Bruce becoming Batman for the first time.


While not the most popular version of Batman (I didn’t even know the show had Batman in it until doing research for this article), it promises to give a good origin story, and a side of the character we haven’t seen before.

The Batman (2022), played by Robert Pattinson
Wu23mtvdhzbspep3rpmkrwm3fm - from adam west to robert pattinson: this is how batman has changed over the years
This film, directed by Matt Reeves, is considered a reboot of the Batman film franchise (after the studio didn’t know what to do with Ben Affleck’s version).

You might find interesting: Robert Pattinson’s must watch films from ‘Twilight’ to ‘The Batman’

It tells the story of a young Batman, who has only been fighting crime for two years and is uncovering corruption while pursuing the Riddler, a serial killer who targets Gotham’s elite.

Perhaps you have already seen it, or perhaps you’re like me and haven’t been able to find the time. Either way, the memes for this movie and the use of Nirvana’s Something in the Way have been incredible.
"""

chunks = chunk_text(text, window_size=4, stride=1)
print(len(chunks))
chunks

10


['From Adam West to Robert Pattinson: this is how Batman has changed over the years\nBatman is one of the most recognizable superheroes in pop culture, starting with his first appearance in Detective Comics #27 in 1939. He has been listed as one of the most iconic characters in popular culture, and among the greatest comic book superheroes and fictional characters ever created. While Batman is sometimes paired with Superman,\n\nIsabel Cara\n\n00:08 / 00:20\nBatman is one of the most recognizable superheroes in pop culture, starting with his first appearance in Detective Comics #27 in 1939. He has been listed as one of the most iconic characters in popular culture, and among the greatest comic book superheroes and fictional characters ever created.\n\nWhile Batman is sometimes paired with Superman, Wonder Woman, and others from the Justice League, his stories have so much potential that they have been made into movies, tv shows, videogames, toy lines, radio, even musical theatre.\n\nYou

In [6]:
import numpy as np
import faiss

# Get vector embeddings for the chunks
vectors = np.array([get_vector_embeddings(chunk) for chunk in chunks])
print(vectors.shape)

# Create a FAISS index
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

# Function to perform a vector search
def vector_search(query_text, k=3):
    query_vector = get_vector_embeddings(query_text)
    distances, indices = index.search(np.array([query_vector]), k)
    return [(chunks[i], float(dist)) for dist, i in zip(distances[0], indices[0])]

# Example search
search_results = vector_search("robert pattinson")
print("Search results for 'lorem ipsum':")
for result in search_results:
    print(result)


(10, 1536)
Search results for 'lorem ipsum':
('Nevertheless, this movie tends to be ignored currently, with many fans not even remembering that Val Kilmer was Batman.\n\nBatman & Robin (1997), played by George Clooney\n3bulsal75ncwzny3xvd2nz33le - from adam west to robert pattinson: this is how batman has changed over the years\nThis film is regarded as the worst Batman movie and even getting spots as one of the worst movies ever made.\n\n\nIt has become infamous because of the neon design, the Batnipples, and just being a franchise-killing movie (after being considered a box office disappointment, Warner Bros. decided to cancel future Batman films, like Schumacher’s -the director of this movie and Batman Forever- Batman Unchained).\n\nIt’s not even considered by fans “so bad it’s good”, and it’s almost always skipped in rewatches of the films.\n\nBatman Begins (2005), played by Christian Bale\nDoggryf3mbfklnwfyzr2t7vyoi - from adam west to robert pattinson: this is how batman has chan

In [7]:
# Function to perform a vector search and then ask GPT-3.5-turbo a question
def search_and_chat(chat_prompt, k=3):
  # Perform the vector search
  search_results = vector_search(chat_prompt, k)
  print(f"Search results: {search_results}\n\n")

  prompt_with_context = f"""Context:{search_results}\
  Answer the question: {chat_prompt}"""

  # Create a list of messages for the chat
  messages = [
      {"role": "system", "content": "Please answer the questions provided by the user. Use only the context provided to you, if you don't know the answer say \"I don't know\"."},
      {"role": "user", "content": prompt_with_context},
  ]

  # Get the model's response
  response = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)

  # Print the assistant's reply
  print(f"Response: {response.choices[0].message.content}")

# Example search and chat
search_and_chat("What song is associated with Robert Pattinson's Batman?")

Search results: [('The film wanted to create an emotional connection with the audience and the Batman and Bruce Wayne identities.\n\nThe film became the 9th highest-grossing film of 2005 and was followed by The Dark Knight (2008), and The Dark Knight Rises (2012). Creating The Dark Knight Trilogy, and the reason many DC fans say that DC movies are better than Marvel Films.\n\nBatman v Superman: Dawn of Justice (2016), played by Ben Affleck\nR6tzxansdngnra5ee5izdeocva - from adam west to robert pattinson: this is how batman has changed over the years\nAffleck is the only actor (so far) to not get his solo movie, and the possibilities of it happening are uncertain, maybe even impossible.\n\nNevertheless, his role is different from the other Batmans, with him being older and jaded because of all the deaths and destruction he has seen but being hopeful in humanity among the member of the Justice League.\n\nHe feels like Batman and Bruce are the same guys, and it really is a shame we couldn